In [187]:
% run contactsScraper.py


Done
Done
KEYS
Done
Done
RECORDS COLLECTED
DATAFRAMES READY


In [2]:
a = OrgSession(orgRecords)

In [3]:
a1 = a.processSession('National Association for Multi-Ethnicity In Communications (NAMIC)')

In [4]:
a.anyQueryTimeouts()

False

In [5]:
a.orgSessionStatusCheck()

['Tue Oct 03, 2017  10:13:50', '12.554', 's', 'Good']

In [6]:
a.orgSessionStatusCheck()

['Tue Oct 03, 2017  10:13:50', '12.554', 's', 'Good']

In [7]:
a1

In [8]:
namicQuery = a1[0]

In [9]:
namicQuery

In [10]:
org = 'National Association for Multi-Ethnicity In Communications (NAMIC)'

In [223]:
def commonParent(A,B):
    #step 1 A to B's Parents
    bParents = B.find_parents()
    if A in bParents:
        return A
    
    #step 2 B to A's Parents
    aParents = A.find_parents()
    if B in aParents:
        return B
    
    Parent_of_b = B.parents
    Parent_of_a = A.parents
    
    #Step 3 Find Firstof A's ancestors to match in B's
    aFirstCommonParent = next(Parent_of_a)
    while aFirstCommonParent not in bParents:
        aFirstCommonParent = next(Parent_of_a)
    
    #Step 4 Find of First of B's ancestors to match in A's
    bFirstCommonParent = next(Parent_of_b)
    while bFirstCommonParent not in aParents:
        bFirstCommonParent = next(Parent_of_b)

    #Step 5 do they match    
    if bFirstCommonParent is aFirstCommonParent:
        return bFirstCommonParent
    else:
        return None    

## Start Here

In [188]:
contacts = cr[cr["Account Name"] == org]

In [189]:
contacts

,Account ID,Account Name,Contact ID,Contact Source,Email,First Name,Last Name,Mailing City,Mailing Country,Mailing State/Province,Mailing Street,Mailing Zip/Postal Code,Notes,Phone,Status,Title
3450,0010B00001liAqw,National Association for Multi-Ethnicity In Co...,0030B00001ueBfz,http://namic.com/the-team/,susan.waldman@namic.com,Susan,Waldman,New York,,NY,"50 Broad Street, Suite 1801",10004,,(646) 545-2508,,"Senior Manager, Membership and Publications"
3451,0010B00001liAqw,National Association for Multi-Ethnicity In Co...,0030B00001ueBhC,http://namic.com/the-team/,monique.wells@namic.com,Monique,Wells,New York,,NY,"50 Broad Street, Suite 1801",10004,,(646) 545-2509,,Director of Finance and Administration
3452,0010B00001liAqw,National Association for Multi-Ethnicity In Co...,0030B00001ueBew,http://namic.com/the-team/,darryl.jackson@namic.com,Darryl,Jackson,New York,,NY,"50 Broad Street, Suite 1801",10004,,(646) 545-2507,,"Manager, Mentoring Relations & Chapter Operations"


In [13]:
namicSoup = namicQuery.get_soup()

In [14]:
namicSource = namicQuery.get_pageSource()

In [16]:
import re

In [190]:
len(contacts)

3

## Pull Record Row

In [284]:
rec = contacts.iloc[[2]]

In [285]:
rec

,Account ID,Account Name,Contact ID,Contact Source,Email,First Name,Last Name,Mailing City,Mailing Country,Mailing State/Province,Mailing Street,Mailing Zip/Postal Code,Notes,Phone,Status,Title
3452,0010B00001liAqw,National Association for Multi-Ethnicity In Co...,0030B00001ueBew,http://namic.com/the-team/,darryl.jackson@namic.com,Darryl,Jackson,New York,,NY,"50 Broad Street, Suite 1801",10004,,(646) 545-2507,,"Manager, Mentoring Relations & Chapter Operations"


## Set Regex for LastName, FirstName and Title

In [286]:
firstNameReg = re.compile('^.*%s.*$' % rec['First Name'].to_string(index=False))
lastNameReg = re.compile('^.*%s.*$' % rec['Last Name'].to_string(index=False))
titleReg = re.compile('^.*%s.*$' % rec['Title'].to_string(index=False))
managerReg = re.compile('^.*Manager.*$')

## Find Fred and Larry, Elements in tree that match text for LastName, FirstName and Title

In [287]:
contactPointers = {}

In [288]:
fred = namicSoup.findAll(text=firstNameReg)

In [289]:
contactPointers['fred'] = fred[0]

In [290]:
larry = namicSoup.findAll(text=lastNameReg)

In [291]:
contactPointers['larry'] = larry[0]

In [292]:
contactPointers

{'fred': 'Darryl Jackson', 'larry': 'Darryl Jackson'}

## Evaluate a bunch of Reggies to find the one that is Tom.  Match Titles Word for Word to Identify title element

In [293]:
titleWords = rec['Title'].to_string(index=False).split()

In [294]:
reggieCounts = {}
for t in titleWords:
    wordReg = re.compile('^.*%s.*$' % t)
    reggies = namicSoup.findAll(text=wordReg)
    for reggie in reggies:
        if reggie in reggieCounts:
            reggieCounts[reggie] += 1
        else:
            reggieCounts[reggie] = 1
    print(t+': '+str(reggies))

Manager,: ['Senior Manager, Membership and Publications', 'Manager, Mentoring Relations and Chapter Operations']
Mentoring: ['Mentoring Corner Webinars', 'Manager, Mentoring Relations and Chapter Operations']
Relations: ['Vice President, Events and Partner Relations', 'Manager, Mentoring Relations and Chapter Operations']
&: ['Membership & Chapters', 'Events & News', 'VIEW ALL SPONSORS & PARTNERS', 'Events & News']
Chapter: ['Membership & Chapters', 'NAMIC Chapters', 'Chapter Elections', 'Manager, Mentoring Relations and Chapter Operations', 'Chapters']
Operations: ['Manager, Mentoring Relations and Chapter Operations']


In [295]:
reggieCounts

{'Chapter Elections': 1,
 'Chapters': 1,
 'Events & News': 2,
 'Manager, Mentoring Relations and Chapter Operations': 5,
 'Membership & Chapters': 2,
 'Mentoring Corner Webinars': 1,
 'NAMIC Chapters': 1,
 'Senior Manager, Membership and Publications': 1,
 'VIEW ALL SPONSORS & PARTNERS': 1,
 'Vice President, Events and Partner Relations': 1}

In [296]:
#maxElem = list(reggieCounts.keys())
#maxElem = reggie
reggieMax = list(reggieCounts.keys())[0]

for reggie in reggieCounts:
    if reggieCounts[reggie] > reggieCounts[reggieMax]:
        reggieMax = reggie

reggieMax

'Manager, Mentoring Relations and Chapter Operations'

In [297]:
contactPointers['tom'] = reggieMax

In [298]:
contactPointers

{'fred': 'Darryl Jackson',
 'larry': 'Darryl Jackson',
 'tom': 'Manager, Mentoring Relations and Chapter Operations'}

## Find Nathan or Nate

In [299]:
if contactPointers['fred'] is contactPointers['larry']:
    ## Most Name Pointers will be nathan(s) because the first and last name of a contact will be in the same element
    contactPointers['nathan'] = contactPointers['fred']
else:
    contactPointers['nate'] = commonParent(contactPointers['fred'], contactPointers['larry'])

In [300]:
contactPointers

{'fred': 'Darryl Jackson',
 'larry': 'Darryl Jackson',
 'nathan': 'Darryl Jackson',
 'tom': 'Manager, Mentoring Relations and Chapter Operations'}

## Assign Find Mother Element shes either Mindy, Martina, or the general case Mary

In [301]:
if 'nathan' in contactPointers:
    if contactPointers['nathan'] is contactPointers['tom']:
        ## Mindy: All in one
        contactPointers['mindy'] = contactPointers['nathan']
    else:
        ## Mary: the General Case
        contactPointers['mary'] = commonParent(contactPointers['nathan'], contactPointers['tom'])

elif 'nate' in contactPointers:
    tomsCommonParent = commonParent(contactPointers['nate'], contactPointer['tom'])
    if tomsCommonParent is contactPointers['nate']:
        ## Martina: nate is toms parent too
        contactPointers['martina'] = tomsCommonParent
    else:
        ## Mary: again, the General Case
        contactPointers['mary'] = tomsCommonParent

In [302]:
contactPointers

{'fred': 'Darryl Jackson',
 'larry': 'Darryl Jackson',
 'mary': <div class="team-member-right"><h3 class="team-member-name">Darryl Jackson</h3><h4 class="team-member-position">Manager, Mentoring Relations and Chapter Operations</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Manager for Mentoring Relations and Chapter Operations, Darryl oversees the day-to-day operations of NAMIC chapters and stewardship of the L. Patrick Mellon Mentorship Program. In this role, Darryl provides programmatic support and guidance in both these key areas, and works with the Senior Vice President of Education and Diversity Solutions to ensure that program participants and volunteers have access to the resources necessary for success in their endeavors, and are able to maximize the benefits of participation and affiliation.
 Darryl began his career in government and finance as a loan officer for the U.S. Department of Agriculture and later moved into mortgage

In [283]:
contactPointers

{'fred': 'S. Monique Wells',
 'larry': 'S. Monique Wells',
 'mary': <div class="team-member-right"><h3 class="team-member-name">S. Monique Wells</h3><h4 class="team-member-position">Director of Finance and Administration</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">Monique Wells is the Director of Finance and Administration for the National Association for Multi-ethnicity in Communications’ (NAMIC). Prior to that she was their Manager of Operations, a position she assumed in August, 2009.
 Monique is no stranger to NAMIC and has been a member since 1991. She served as a consultant to NAMIC for a number of special projects, prior to joining the staff. As Director of Finance and Administration, she oversees the administrative and financial operations of the national office.
 An accomplished professional with a diverse background in sales and project management, Monique previously served as an Energy Conservation Analyst with the Los Angeles Un

In [264]:
contactPointers

{'fred': 'Susan Waldman',
 'larry': 'Susan Waldman',
 'mary': <div class="team-member-right"><h3 class="team-member-name">Susan Waldman</h3><h4 class="team-member-position">Senior Manager, Membership and Publications</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest.
 She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industry.
 Susan has served as

In [151]:
contactPointers

{'fred': 'S. Monique Wells',
 'larry': 'S. Monique Wells',
 'tom': 'Director of Finance and Administration'}

In [121]:
contactPointers

{'fred': 'Darryl Jackson',
 'larry': 'Darryl Jackson',
 'tom': 'Manager, Mentoring Relations and Chapter Operations'}

In [166]:
if contactPointers['fred'] is contactPointers['larry']:
    contactPointers['nathan'] = contactPointers['fred']
else
    

True

In [168]:
contactPointers['fred'].find_parents()

[<h3 class="team-member-name">Susan Waldman</h3>,
 <div class="team-member-right"><h3 class="team-member-name">Susan Waldman</h3><h4 class="team-member-position">Senior Manager, Membership and Publications</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest.
 She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industry.
 Susan has served as the Market

In [169]:
contactPointers['tom'].find_parents()

[<h4 class="team-member-position">Senior Manager, Membership and Publications</h4>,
 <div class="team-member-right"><h3 class="team-member-name">Susan Waldman</h3><h4 class="team-member-position">Senior Manager, Membership and Publications</h4><div class="team-member-body hide-class"><div style="overflow: hidden; height: 72px;">As the Senior Manager of Membership and Publications, Susan Waldman oversees the day-to-day facilitation of NAMIC’s membership operations, as well as market research to drive growth. She is also responsible for the production of NAMIC’s industry-wide publication, Diversity Digest.
 She launched her career in market research as a media planner on such accounts as Loews Hotels and Omnipoint Communications at Hill, Holliday Altschiller and McCann Direct advertising agencies.  There she was introduced to the gamut of advertising research resources as she planned and purchased the media for various clients.  After a few years, Susan moved over to the magazine industr

In [205]:
tomsParents = contactPointers['tom'].parents

In [218]:
next(tomsParents)

StopIteration: 

In [224]:
contactPointers

{'fred': 'Susan Waldman',
 'larry': 'Susan Waldman',
 'tom': 'Senior Manager, Membership and Publications'}

In [225]:
commonParent(contactPointers['fred'],contactPointers['larry'])

<h3 class="team-member-name">Susan Waldman</h3>

In [227]:
testA = commonParent(contactPointers['tom'],contactPointers['larry'])

In [229]:
testB = commonParent(contactPointers['larry'],contactPointers['tom'])

In [230]:
testB is testA

True

In [17]:
reg = re.compile('^.*Eglon.*$')

In [18]:
namicSoup.findAll(text=reg)

['Eglon E. Simons']

In [19]:
namicSoup.findAll(text=reg)

['Eglon E. Simons']

In [20]:
namicSoup.findAll(text=reg)[0].parent

<h3 class="team-member-name">Eglon E. Simons</h3>

In [21]:
namicSoup(text=reg)

['Eglon E. Simons']

In [22]:
namicSoup(text=reg)[0].parent

<h3 class="team-member-name">Eglon E. Simons</h3>

In [23]:
titleReg = re.compile('^.*President and CEO.*$')

In [24]:
namicSoup(text=titleReg)

['President and CEO']

In [25]:
namicSoup(text=titleReg)[0].parent

<h4 class="team-member-position">President and CEO</h4>

In [26]:
type(namicSoup(text=titleReg)[0])

bs4.element.NavigableString

In [27]:
type(namicSoup(text=titleReg)[0].parent)

bs4.element.Tag

In [29]:
namicSoup(text=titleReg)[0].parent.parent.name

'div'

In [38]:
type(namicSoup(text=titleReg)[0].parent.parent)

bs4.element.Tag

In [44]:
firstNameReg = re.compile('^.*James.*$')
lastNameReg = re.compile('^.*Jones.*$')
altFirstNameReg = re.compile('^.*Eglon.*$')

In [48]:
firstNameElm = namicSoup(text=firstNameReg)[0]
lastNameElm = namicSoup(text=lastNameReg)[0]
altFirstNameElm = namicSoup(text=altFirstNameReg)[0]

In [49]:
firstNameElm is lastNameElm

True

In [50]:
altFirstNameElm is lastNameElm

False